In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras import Model

mnist=tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [3]:
x_train=x_train[...,tf.newaxis].astype("float32")
x_test=x_test[...,tf.newaxis].astype("float32")
# 原本是n*28*28，在最后加了一个轴，相当于给每一个数加了一个括号，即变成了n*28*28*1

In [4]:
train_ds=tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)).shuffle(10000).batch(32)

test_ds=tf.data.Dataset.from_tensor_slices(
    (x_test,y_test)).shuffle(10000).batch(32)

In [5]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1=Conv2D(32,3,activation='relu')
        self.flatten=Flatten()
        self.d1=Dense(128,activation='relu')
        self.d2=Dense(10)

    def call(self,x):
        x=self.conv1(x)
        x=self.flatten(x)
        x=self.d1(x)
        return self.d2(x)

model=MyModel()
loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer=tf.keras.optimizers.Adam()

In [6]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions=model(images,training=True)
        loss=loss_object(labels,predictions) #计算出损失
    gradients = tape.gradient(loss, model.trainable_variables)#根据损失计算导数
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))#根据导数以及优化方案，将导数与变量绑定在一起然后进行梯度下降
    #这样设计不错奥
    train_loss(loss)
    train_accuracy(labels,predictions)

@tf.function #将这个方法改造为graph中的一个op
def test_step(images,labels):
    predictions=model(images,training=False)
    t_loss=loss_object(labels,predictions)
    test_loss(t_loss)
    test_accuracy(labels,predictions)

In [8]:
EPOCHS=5
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images,labels in train_ds:
        train_step(images,labels)

    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

# 从上面的例子可以看出，model的__call__函数与各个layer的__call__函数对输入的要求相同
# 都得是具体的一些样本，形如[sample_num,feature_num]这类的
# 而不能是一个dataset



Epoch 1, Loss: 0.14318040013313293, Accuracy: 95.71499633789062, Test Loss: 0.05952138453722, Test Accuracy: 98.02999877929688
Epoch 2, Loss: 0.042943283915519714, Accuracy: 98.66166687011719, Test Loss: 0.05447855219244957, Test Accuracy: 98.20999908447266
Epoch 3, Loss: 0.02336840145289898, Accuracy: 99.26666259765625, Test Loss: 0.054638300091028214, Test Accuracy: 98.40999603271484
Epoch 4, Loss: 0.014808130450546741, Accuracy: 99.49166870117188, Test Loss: 0.05886276438832283, Test Accuracy: 98.47000122070312
Epoch 5, Loss: 0.009999511763453484, Accuracy: 99.66666412353516, Test Loss: 0.06575784832239151, Test Accuracy: 98.29000091552734
